
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel provincial de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

# Webscraping

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Chrome


from time import sleep
# Selenium para establecer la configuración del driver
# -----------------------------------------------------------------------
from selenium import webdriver

# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os

import re

import zipfile
import shutil

import pandas as pd
import requests
import json

/Users/davidfranco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [41]:
## Opción accediendo directamente a la descarga

""" # lo primero que vamos a hacer es configurar nuestras preferencias del navegador para el driver
chrome_options = webdriver.ChromeOptions()


# establacemos las preferencias que queremos
prefs = {
    "download.default_directory": "/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosDescargados",  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}

url = "https://www.ine.es/jaxi/Tabla.htm?path=/t20/e245/p08/l0/&file=03001.px&L=0"

# ahora debo pasar mi diccionario de preferencias
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options= chrome_options)
actions = ActionChains(driver)

driver.get(url)
sleep(2)
driver.find_element("css selector", "#btnDescarga > i").click()
sleep(2)
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", '//*[@id="thickBoxINEfrm"]')))

driver.switch_to.frame(iframe)

try: #ahora is podemos buscar botón
    driver.find_element("css selector", "body > ul > li:nth-child(4) > a").click()

except:
    print("No encuentro botón")

driver.quit() """

In [13]:
df_poblacion = pd.read_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosDescargados/poblacion.csv", sep=';', encoding='utf-8')

In [5]:
df_poblacion.head(1)

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2022,47.475.420


In [30]:
df_sucio1 = df_poblacion[df_poblacion["Año"].isin([2019,2020,2021])]

In [31]:
df_sucio1

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
1,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2021,47.385.107
2,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2020,47.450.795
3,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2019,47.026.208
26,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Hombres,2021,23.222.953
27,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Hombres,2020,23.255.590
...,...,...,...,...,...,...
63552,52 Melilla,65 y más,% Extranjeros,Hombres,2020,"10,86"
63553,52 Melilla,65 y más,% Extranjeros,Hombres,2019,"10,73"
63576,52 Melilla,65 y más,% Extranjeros,Mujeres,2021,"9,55"
63577,52 Melilla,65 y más,% Extranjeros,Mujeres,2020,"9,55"


In [34]:
sucio2 = df_poblacion[
    (df_poblacion['Edad (3 grupos de edad)'] != 'TOTAL EDADES') & 
    (df_poblacion['Españoles/Extranjeros'] != 'TOTAL') & 
    (df_poblacion['Provincias'] != 'TOTAL ESPAÑA')
]

In [35]:
sucio2

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
1575,02 Albacete,0-15 años,Españoles,Ambos sexos,2022,51.924
1576,02 Albacete,0-15 años,Españoles,Ambos sexos,2021,52.851
1577,02 Albacete,0-15 años,Españoles,Ambos sexos,2020,53.932
1578,02 Albacete,0-15 años,Españoles,Ambos sexos,2019,55.028
1579,02 Albacete,0-15 años,Españoles,Ambos sexos,2018,55.893
...,...,...,...,...,...,...
63595,52 Melilla,65 y más,% Extranjeros,Mujeres,2002,"5,13"
63596,52 Melilla,65 y más,% Extranjeros,Mujeres,2001,"4,65"
63597,52 Melilla,65 y más,% Extranjeros,Mujeres,2000,"3,34"
63598,52 Melilla,65 y más,% Extranjeros,Mujeres,1999,"0,81"


In [7]:
poblacion = "https://www.ine.es/jaxi/Tabla.htm?path=/t20/e245/p08/l0/&file=03001.px&L=0"
ipc = "https://www.ine.es/jaxi/Tabla.htm?tpx=67284&L=0"
links = [poblacion, ipc]


for i in range(len(links)):

    chrome_options = webdriver.ChromeOptions()
    prefs = {
        "download.default_directory": "/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosDescargados",
        "download.prompt_for_download": False,
        "directory_upgrade": True,
    }
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    sleep(1)
    driver.get(links[i])
    sleep(1)
    
    driver.find_element("css selector", "#tg0 > div > fieldset > div.capaSelecTodosNinguno > button.opcionesvarDer > i").click()
    sleep(1)
    driver.find_element("css selector", "#tg1 > div > fieldset > div.capaSelecTodosNinguno > button.opcionesvarDer > i").click()
    sleep(1)
    if i == 0:
        driver.find_element("css selector", "#tg2 > div > fieldset > div.capaSelecTodosNinguno > button.opcionesvarDer > i").click()
        sleep(1)
        driver.find_element("css selector", "#tg3 > div > fieldset > div.capaSelecTodosNinguno > button.opcionesvarDer > i").click()
        sleep(1)
    
    # Seleccionar años
    driver.find_element("css selector", "#caja_periodo > div > fieldset > div.capaSelecTodosNinguno > button:nth-child(3) > i").click()
    sleep(1)
    
    if i == 0:
        yr1 = driver.find_element("css selector", "#periodo > option:nth-child(4)")
    else:
        yr1 = driver.find_element("css selector", "#periodo > option:nth-child(1)")
    
    yr2 = driver.find_element("css selector", "#periodo > option:nth-child(3)")
    yr3 = driver.find_element("css selector", "#periodo > option:nth-child(2)")

    years_ine = [yr1, yr2, yr3]
    sleep(1)

    actions = ActionChains(driver)
    actions.key_down(Keys.COMMAND)

    for year in years_ine:
        actions.click(year)

    actions.key_up(Keys.COMMAND)
    actions.perform()
    
    driver.execute_script("window.scrollTo(0, 1000);")
    driver.find_element("css selector", "#botonConsulSele").click()
    sleep(1)
    driver.find_element("css selector", "#btnDescargaForm > i").click()
    sleep(1)
    
    iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("css selector", '#thickBoxINEfrm')))
    driver.switch_to.frame(iframe)
    sleep(2)
    driver.find_element("css selector", "#export > ul > li:nth-child(4) > label").click()

    driver.quit()


In [12]:
# Renombrar

path = "/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosDescargados/"

for file in os.listdir(path):

    if file == "03001.csv":
        os.rename(f"{path}/{file}", f"{path}/poblacion.csv")

    if file == "67284.csv":
        os.rename(f"{path}/{file}", f"{path}/pib.csv")

# API Red Eléctrica

In [87]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

### API Red Eléctrica Evolución

In [112]:
years = [2019, 2020, 2021]

for year in years:
    for comunidad, id in cod_comunidades.items():
        url = f"https://apidatos.ree.es/es/datos/demanda/evolucion?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=month&geo_id={id}"
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Host": "apidatos.ree.es"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            file = f"/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/evolucion/{year}"
            os.makedirs(file, exist_ok=True)
            nombre = comunidad + ".json"
            ruta_archivo = os.path.join(file, nombre)
            with open(ruta_archivo, "w") as archivo_json:
                json.dump(response.json(), archivo_json, indent=4)
        else:
            print(f"Error en la comunidad {comunidad} para el año {year} por motivo {response.status_code}")


### API Red Eléctrica Generación

In [114]:
for year in years:
    for comunidad, id in cod_comunidades.items():
        url = f"https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date={year}-01-01T00:00&end_date={year}-12-31T23:59&time_trunc=month&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={id}"
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json",
            "Host": "apidatos.ree.es"
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            file = f"/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/generacion/{year}"
            os.makedirs(file, exist_ok=True)
            nombre = comunidad + ".json"
            ruta_archivo = os.path.join(file, nombre)
            with open(ruta_archivo, "w") as archivo_json:
                json.dump(response.json(), archivo_json, indent=4)
        else:
            print(f"Error en la comunidad {comunidad} para el año {year} por motivo {response.status_code}")


Error en la comunidad Ceuta para el año 2019 por motivo 502
Error en la comunidad Ceuta para el año 2020 por motivo 502
Error en la comunidad Ceuta para el año 2021 por motivo 502
